In [11]:
from langchain_core.messages import (ToolMessage)
from langchain_core.tools import BaseTool, tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from typing import (
    Any,
    AsyncIterator,
    Callable,
    Dict,
    Iterator,
    List,
    Literal,
    Mapping,
    Optional,
    Sequence,
    Tuple,
    Type,
    TypedDict,
    TypeVar,
    Union,
    cast,
    overload,
)

In [3]:
# Tool definition
@tool
def MyTool(text):
    """My Tool"""
    # Simulate processing time
    import time
    time.sleep(1)
    # Generate a random response
    import random
    responses = ['Response 1', 'Response 2', 'Response 3']
    return random.choice(responses)

In [4]:
MyTool.invoke("Hello")

'Response 3'

In [6]:
tools=[MyTool]

In [23]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("TAVILY_API_KEY")

In [24]:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
tool.invoke("What's a 'node' in LangGraph?")

[{'url': 'https://langchain-ai.github.io/langgraph/concepts/',
  'content': "If a node has multiple out-going edges, all of those destination nodes will be executed in parallel as a part of the next superstep. State Management¶ LangGraph introduces two key ideas to state management: state schemas and reducers. The state schema defines the type of the object that is given to each of the graph's Node."},
 {'url': 'https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141',
  'content': 'Nodes: Nodes are the building blocks of your LangGraph. Each node represents a function or a computation step. You define nodes to perform specific tasks, such as processing input, making ...'}]

In [75]:
tools = [tool]

In [76]:
formatted_tools = [convert_to_openai_tool(tool)["function"] for tool in tools]

In [77]:
formatted_tools

[{'name': 'tavily_search_results_json',
  'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
  'parameters': {'type': 'object',
   'properties': {'query': {'description': 'search query to look up',
     'type': 'string'}},
   'required': ['query']}}]

In [49]:
formatted_tools

[{'name': 'tavily_search_results_json',
  'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
  'parameters': {'type': 'object',
   'properties': {'query': {'description': 'search query to look up',
     'type': 'string'}},
   'required': ['query']}}]

In [28]:
_dict=formatted_tools[0]

In [29]:
additional_kwargs = {}
if "name" in _dict:
    additional_kwargs["name"] = _dict["name"]
name = _dict.get("name")    

In [50]:
ToolMessage(
            #content=_dict.get("content", ""),
            content=_dict.get("description", ""),
            tool_call_id=1, #cast(str, _dict.get("tool_call_id")),
            additional_kwargs=additional_kwargs,
            name=name,
            #id=id_,
        )

ToolMessage(content='A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.', additional_kwargs={'name': 'tavily_search_results_json'}, name='tavily_search_results_json', tool_call_id='1')

In [88]:
message = ToolMessage(
            content=_dict.get("content", ""),
            #content=_dict.get("description", ""),
            tool_call_id=1, #cast(str, _dict.get("tool_call_id")),
            additional_kwargs=additional_kwargs,
            name=name,
            #id=id_,
        )

In [79]:
generation_info ={"finish_reason": "Invocation Tool"} 

In [80]:
from langchain_core.outputs import ChatGeneration, ChatGenerationChunk, ChatResult


In [81]:
gen = ChatGeneration(
    message=message,
    generation_info=generation_info,
)

In [82]:
generations = []

In [83]:
generations.append(gen)

In [68]:
llm_output = {
    "token_usage": 0,
    "model_name": "test",
    "system_fingerprint": "test",
}

In [84]:
ChatResult(generations=generations, llm_output=llm_output)

ChatResult(generations=[ChatGeneration(generation_info={'finish_reason': 'Invocation Tool'}, message=ToolMessage(content='', additional_kwargs={'name': 'tavily_search_results_json'}, name='tavily_search_results_json', tool_call_id='1'))], llm_output={'token_usage': 0, 'model_name': 'test', 'system_fingerprint': 'test'})

In [85]:
chat_result=ChatResult(generations=generations, llm_output=llm_output)

In [86]:
# Print the generated text
for generation in chat_result.generations:
    print(generation.text)

In [87]:
# Print the raw LLM output (if you need to inspect it)
print(chat_result.llm_output)

{'token_usage': 0, 'model_name': 'test', 'system_fingerprint': 'test'}


In [ ]:
def _convert_dict_to_message(_dict: Mapping[str, Any]) -> BaseMessage:
    """Convert a dictionary to a LangChain message.

    Args:
        _dict: The dictionary.

    Returns:
        The LangChain message.
    """
    role = _dict.get("role")
    name = _dict.get("name")
    id_ = _dict.get("id")
    if role == "user":
        return HumanMessage(content=_dict.get("content", ""), id=id_, name=name)
    elif role == "assistant":
        # Fix for azure
        # Also OpenAI returns None for tool invocations
        content = _dict.get("content", "") or ""
        additional_kwargs: Dict = {}
        if function_call := _dict.get("function_call"):
            additional_kwargs["function_call"] = dict(function_call)
        tool_calls = []
        invalid_tool_calls = []
        if raw_tool_calls := _dict.get("tool_calls"):
            additional_kwargs["tool_calls"] = raw_tool_calls
            for raw_tool_call in raw_tool_calls:
                try:
                    tool_calls.append(parse_tool_call(raw_tool_call, return_id=True))
                except Exception as e:
                    invalid_tool_calls.append(
                        make_invalid_tool_call(raw_tool_call, str(e))
                    )
        return AIMessage(
            content=content,
            additional_kwargs=additional_kwargs,
            name=name,
            id=id_,
            tool_calls=tool_calls,
            invalid_tool_calls=invalid_tool_calls,
        )
    elif role == "system":
        return SystemMessage(content=_dict.get("content", ""), name=name, id=id_)
    elif role == "function":
        return FunctionMessage(
            content=_dict.get("content", ""), name=cast(str, _dict.get("name")), id=id_
        )
    elif role == "tool":
        additional_kwargs = {}
        if "name" in _dict:
            additional_kwargs["name"] = _dict["name"]
        return ToolMessage(
            content=_dict.get("content", ""),
            tool_call_id=cast(str, _dict.get("tool_call_id")),
            additional_kwargs=additional_kwargs,
            name=name,
            id=id_,
        )
    else:
        return ChatMessage(content=_dict.get("content", ""), role=role, id=id_)

In [91]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import BaseTool, tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.outputs import ChatGeneration, ChatResult
from typing import Any, Dict

def generate_tool_output(tool_instance, input_text: str, max_results: int = 2)-> ChatResult:
    """
    Generate chatbot output using the given tool and input text.
    Args:
    - tool_instance
    - input_text (str): Input text to process
    Returns:
    - ChatResult: The generated chatbot output
    """
    # Invoke the tool with the input text
    tool_instance.invoke(input_text)
    # Convert the tool to an OpenAI tool
    formatted_tool = convert_to_openai_tool(tool_instance)["function"]
    # Create a ToolMessage instance
    message = ToolMessage(
        content=formatted_tool.get("content", ""),
        tool_call_id=1,
        additional_kwargs={"name": formatted_tool.get("name")},
        name=formatted_tool.get("name"),
    )
    # Create a ChatGeneration instance
    generation_info = {"finish_reason": "Invocation Tool"}
    gen = ChatGeneration(message=message, generation_info=generation_info)
    # Create a list of generations
    generations = [gen]
    # Create an LLM output instance
    llm_output = {
        "token_usage": 0,
        "model_name": "test",
        "system_fingerprint": "test",
    }
    # Create a ChatResult instance
    chat_result = ChatResult(generations=generations, llm_output=llm_output)
    return chat_result
# Example usage:
from langchain_community.tools.tavily_search import TavilySearchResults
tool = TavilySearchResults(max_results=2)
input_text = "What's a 'node' in LangGraph?"
chat_result = generate_tool_output(tool, input_text)
# Print the generated text
for generation in chat_result.generations:
    print(generation.text)
# Print the raw LLM output (if you need to inspect it)
print(chat_result.llm_output)


{'token_usage': 0, 'model_name': 'test', 'system_fingerprint': 'test'}


In [92]:
from langchain_core.messages import ToolMessage
from langchain_core.tools import BaseTool, tool
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.outputs import ChatGeneration, ChatResult
from typing import Any, Dict

def generate_tool_output(tool_instance: BaseTool, input_text: str, max_results: int = 2) -> ChatResult:
    """
    Generate chatbot output using the given tool and input text.
    Args:
    - tool_instance: The instance of the tool to use.
    - input_text (str): Input text to process.
    Returns:
    - ChatResult: The generated chatbot output.
    """
    # Invoke the tool with the input text and capture the result
    result = tool_instance.invoke(input_text)

    # Convert the tool to an OpenAI tool
    formatted_tool = convert_to_openai_tool(tool_instance)["function"]

    # Extract the content from the result to include in the ToolMessage
    content = "\n\n".join([f"URL: {item['url']}\nContent: {item['content']}" for item in result])

    # Create a ToolMessage instance
    message = ToolMessage(
        content=content,
        tool_call_id=1,
        additional_kwargs={"name": formatted_tool.get("name")},
        name=formatted_tool.get("name"),
    )

    # Create a ChatGeneration instance
    generation_info = {"finish_reason": "Invocation Tool"}
    gen = ChatGeneration(message=message, generation_info=generation_info)

    # Create a list of generations
    generations = [gen]

    # Create an LLM output instance
    llm_output = {
        "token_usage": 0,
        "model_name": "test",
        "system_fingerprint": "test",
    }

    # Create a ChatResult instance
    chat_result = ChatResult(generations=generations, llm_output=llm_output)

    return chat_result

# Example usage:
from langchain_community.tools.tavily_search import TavilySearchResults

tool = TavilySearchResults(max_results=2)
input_text = "What's a 'node' in LangGraph?"
chat_result = generate_tool_output(tool, input_text)

# Print the generated text
for generation in chat_result.generations:
    print(generation.message.content)

# Print the raw LLM output (if you need to inspect it)
print(chat_result.llm_output)


URL: https://medium.com/@cplog/introduction-to-langgraph-a-beginners-guide-14f9be027141
Content: Step 1: Define the Graph State. First, we define the state structure for our graph. In this example, our state includes the user's question, the classification of the question, and a response ...

URL: https://langchain-ai.github.io/langgraph/concepts/
Content: LangGraph's underlying graph algorithm uses message passing to define a general program. When a Node completes, it sends a message along one or more edges to other node (s). These nodes run their functions, pass the resulting messages to the next set of nodes, and on and on it goes. Inspired by Pregel, the program proceeds in discrete "super-steps" that are all executed conceptually in ...
{'token_usage': 0, 'model_name': 'test', 'system_fingerprint': 'test'}


### Converting and Evaluating Tools in OpenAI Format

Converting tools to the OpenAI format and evaluating them is a useful process when you want to leverage the standardized API for integrating tools with OpenAI models. This is necessary for compatibility and to ensure that tools can be easily called by the model during interaction. The `convert_to_openai_tool` function translates the tool into a format that is recognized by the OpenAI API, allowing it to be utilized effectively within the context of language model interactions.

### Steps for Conversion and Evaluation

1. **Convert the Tool:**
   The `convert_to_openai_tool` function transforms the tool into a standardized dictionary format that OpenAI's API can understand. This includes the tool's name, description, and parameters.

2. **Bind the Tool:**
   Use the `bind_tools` method to attach these tools to the chat model. This prepares the tools to be invoked by the model as needed.

3. **Invoke the Tool:**
   To evaluate and test the tool within the OpenAI format, you need to simulate a call to the tool as if the model is using it.

Let's walk through an example using your provided code.

### Example

```python
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.utils.function_calling import convert_to_openai_tool

# Step 1: Create and convert the tool
tool = TavilySearchResults(max_results=2)
converted_tool = convert_to_openai_tool(tool)

# Step 2: Format the tools for OpenAI
formatted_tools = [converted_tool["function"]]

# Example of formatted_tools
# formatted_tools = [{
#     'name': 'tavily_search_results_json',
#     'description': 'A search engine optimized for comprehensive, accurate, and trusted results. Useful for when you need to answer questions about current events. Input should be a search query.',
#     'parameters': {'type': 'object', 'properties': {'query': {'description': 'search query to look up', 'type': 'string'}}, 'required': ['query']}
# }]

# Step 3: Define a mock chat model (simplified example for illustration)
class MockChatModel:
    def __init__(self, tools):
        self.tools = {tool['name']: tool for tool in tools}

    def call_tool(self, tool_name, params):
        if tool_name not in self.tools:
            raise ValueError(f"Tool {tool_name} not found")
        
        # Simulate tool invocation
        if tool_name == 'tavily_search_results_json':
            # Here you call the original tool's method with params
            query = params['query']
            return tool.invoke(query)  # Original tool invocation
        else:
            raise ValueError(f"Tool {tool_name} not implemented")

# Step 4: Instantiate the chat model with the converted tools
chat_model = MockChatModel(formatted_tools)

# Step 5: Invoke the tool through the chat model
query = "What's a 'node' in LangGraph?"
response = chat_model.call_tool('tavily_search_results_json', {'query': query})
print(response)

```

### Explanation

1. **Conversion:**
   - The `TavilySearchResults` tool is instantiated and then converted to the OpenAI format using `convert_to_openai_tool`.

2. **Formatting:**
   - The formatted tool dictionary is prepared for binding to the chat model.

3. **Mock Chat Model:**
   - A simplified `MockChatModel` is created to simulate tool invocation.
   - This model has a method `call_tool` that takes the tool name and parameters, finds the tool, and calls the original tool's `invoke`

 method with the given parameters.

4. **Invocation:**
   - The tool is invoked by calling `call_tool` on the mock chat model, which simulates how the tool would be used within a language model interaction.

### Why Convert to OpenAI Tool Format?

1. **Standardization:**
   - The OpenAI tool format provides a standardized way to define and use tools, ensuring compatibility and ease of integration with language models.

2. **Interoperability:**
   - By converting tools to this format, they can be easily shared, reused, and combined with other tools in a consistent manner.

3. **Ease of Use:**
   - Tools in the OpenAI format can be invoked programmatically within conversations, enabling seamless interaction and functionality.

### How to Use the Converted Tools

Once the tools are converted and bound to the chat model, you can evaluate their functionality by invoking them as shown in the example. This approach allows you to test the tool's behavior in response to specific inputs and ensure that it performs as expected within the OpenAI tool-calling framework.

By following these steps, you can effectively convert, bind, and evaluate tools within the OpenAI ecosystem, leveraging their standardized API for powerful and flexible interactions.